# Intro
This notebook demonstrates what you can do with the Objectiv Bach modeling library and a dataset that was validated against the open analytics taxonomy. The example uses real data that's stored in an SQL database and was collected with the Objectiv Tracker that's instrumented on objectiv.io.

There is another notebook in the same folder that focuses on using the open model hub [model-hub-demo-notebook.ipynb](model-hub-demo-notebook.ipynb), demonstrating how you can use Bach to stack pre-built models to quickly answer common product analytics questions. 

The Objectiv Bach API is heavily inspired by the pandas API. We believe this provides a great, generic interface to handle large amounts of data in a python environment while supporting multiple data stores.

For an intro into the pandas api see: https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html  
The full Objectiv Bach api reference is available here: https://objectiv.io/docs/modeling/reference

# Contents  
* [Instantiate-the-object](#Instantiate-the-object)
  * [The data](#The-data)
    * [global_contexts & location_stack](#global_contexts-&-location_stack)
    * [location_stack](#location_stack)
    * [global_contexts](#global_contexts)
  * [Sampling](#Sampling)

In [ ]:
from bach_open_taxonomy import ObjectivFrame

# Instantiate the object
As a first step, the Objectiv Frame object is instantiated. The Objectiv Frame is an extension to Objectiv Bach DataFrame to use specifically for data that was collected with Objectiv’s Tracker. It loads the data as stored by the Objectiv Tracker, makes a few transformations, and sets the right data types.

In [ ]:
df = ObjectivFrame.from_objectiv_data(time_aggregation='YYYY-MM-DD')

The data for the DataFrame is still in the database and the database is not queried before any of the data is loaded to the python environment. The methods that query the database are: 
* [`head()`](https://objectiv.io/docs/modeling/dataframe/bach.DataFrame.head#bach.DataFrame.head)
* [`to_pandas()`](https://objectiv.io/docs/modeling/dataframe/bach.DataFrame.to_pandas#bach.DataFrame.to_pandas)
* [`get_sample()`](https://objectiv.io/docs/modeling/dataframe/bach.DataFrame.get_sample#bach.DataFrame.get_sample)
* The property accessors [`values`](https://objectiv.io/docs/modeling/dataframe/bach.DataFrame.values#bach.DataFrame.values), [`Series.array`](https://objectiv.io/docs/modeling/series/bach.Series.array#bach.Series.array), [`Series.value`](https://objectiv.io/docs/modeling/series/bach.Series.value#bach.Series.value)

For demo puposes of this notebook, these methods are called often to show the results of our operations. To limit the number of executed queries on the full data set it is recommended to use these methods less often or [to sample the data first](#sampling).

## The data
The contents of the Objectiv Frame exist of:

In [ ]:
df.index_dtypes

The index contain a unique identifyer for every hit.

In [ ]:
df.dtypes

* `day`: the day of the session as a date.
* `moment`: the exact moment of the event.
* `user_id`: the unique identifyer of the user based on the cookie.
* `session_id`: a unique incremented integer id for each session. Starts at 0 for the selected data in the Objectiv Frame.
* `global_contexts`: a json-like data column that stores additional information on the event that is logged. This includes data like device data, application data, and cookie information. [See below](#global_contexts) for more detailed explanation. 
* `location_stack`: a json-like data column that stores information on the exact location where the event is triggered in the product's UI. [See below](#location_stack) for more detailed explanation.
* `event_type`: the type of event that is logged.
* `stack_event_types`: the parents of the event_type.

A preview of the data:

In [ ]:
df.head()

### global_contexts & location_stack
The global_contexts and location_stack are both columns containing json-like data. Both columns are always arrays of context objects. The json arrays can be accessed with the `.json` accessor. A context object looks like the example below. It _always_ contains a `_type` and `id` key.

In [ ]:
df.location_stack.json[0].head(1)[0]

**Slicing the json data**  
With the `.json[]` syntax you can slice the array using integers. Instead of integers, dictionaries can also be passed to 'query' the json array. If the passed dictionary matches a context object in the stack, all objects of the stack starting (or ending, depending on the slice) at that object will be returned.

**An example**  
Consider a json array that looks like this):
```json
[{'id': '#document', '_type': 'WebDocumentContext'},
 {'id': 'page-about', '_type': 'SectionContext'},
 {'id': 'main', '_type': 'SectionContext'},
 {'id': 'core-team', '_type': 'SectionContext'},
 {'id': 'contributors', '_type': 'SectionContext'},
 {'id': 'jansentom', '_type': 'SectionContext'},
 {'id': 'contributor-card', '_type': 'SectionContext'}]
```
**Regular slicing**
```python
df.location_stack.json[2:4]
```
For the example array it would return:
```json
[{'id': 'main', '_type': 'SectionContext'},
 {'id': 'core-team', '_type': 'SectionContext'}]
```
**Slicing by querying**
We want to return only the part of the array starting at the object that contain this object:
```javascript
{"id": "contributors", "_type": "SectionContext"}
```
The syntax for selecting like this is: 
```python
df.location_stack.json[{"_type": "SectionContext", "id": "contributors"}:]
```
For the example array it would return:
```json
[{'id': 'contributors', '_type': 'SectionContext'},
 {'id': 'jansentom', '_type': 'SectionContext'},
 {'id': 'contributor-card', '_type': 'SectionContext'}]
```
In case a location stack does not contain the object, `None` is returned. More info at the api reference: https://objectiv.io/docs/modeling/series/bach.SeriesJsonb.json#bach.SeriesJsonb.json

### location_stack
The `location_stack` column in an Objectiv Frame that stores the information on the exact location where the event is triggered in the product. The example used above is the location stack of the contributor card of Tom Jansen on this page: https://objectiv.io/about/.

Because of the specific way the location information is labeled, validated, and stored using the Open Taxonomy, it can be used to slice and group your products' features in an efficient and easy way. The column is set as an `objectiv_location_stack` type, and therefore location stack specific methods can be used to access the data from the `location_stack`. These methods can be used using the `.ls` accessor on the column. The methods are:
* The property accessors [`.ls.navigation_features`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.SeriesLocationStack.ls#bach_open_taxonomy.SeriesLocationStack.ls), [`.ls.feature_stack`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.SeriesLocationStack.ls#bach_open_taxonomy.SeriesLocationStack.ls), [`.ls.nice_name`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.SeriesLocationStack.ls#bach_open_taxonomy.SeriesLocationStack.ls)
* all [methods](https://objectiv.io/docs/modeling/series/bach.SeriesJsonb.json#bach.SeriesJsonb.json) for the json(b) type can also be accessed using `.ls`

The full reference of location stack is [here](https://objectiv.io/docs/taxonomy/location-contexts). An example is queried below:

In [ ]:
df.location_stack.head(1)[0]

### global_contexts
The `global_contexts` column in an Objectiv Frame contain all information that is relevant to the logged event. As it is set as an `objectiv_global_context` type, specific methods can be used to access the data from the `global_contexts`. These methods can be used using the `.gc` accessor on the column. The methods are:
* [`.gc.get_from_context_with_type_series(type, key)`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.ObjectivStack#bach_open_taxonomy.ObjectivStack)
* The property accessors [`.gc.cookie_id`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.SeriesGlobalContexts.gc#bach_open_taxonomy.SeriesGlobalContexts.gc), [`.gc.user_agent`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.SeriesGlobalContexts.gc#bach_open_taxonomy.SeriesGlobalContexts.gc), [`.gc.application`](https://objectiv.io/docs/modeling/objectiv/bach_open_taxonomy.SeriesGlobalContexts.gc#bach_open_taxonomy.SeriesGlobalContexts.gc)  
* all [methods](https://objectiv.io/docs/modeling/series/bach.SeriesJsonb.json#bach.SeriesJsonb.json) for the json(b) type can also be accessed using `.gc`

The full reference of global contexts is [here](https://objectiv.io/docs/taxonomy/global-contexts). An example is queried below:

In [ ]:
df.global_contexts.head(1)[0]

# Sampling
One of the key features to Objectiv Bach is that it runs on your full data set. There can however be situations where you want to experiment with your data, meaning you have to query the full data set often. This can become slow and/or costly. 

To limit these costs it is possible to do operations on a sample of the full data set. All operations can easily be applied at any time to the full data set if that is desired.

Below we create a sample that randomly selects ~1% of all the rows in the data. A table containing the sampled is written to the database, therefore the `table_name` must be provided when creating the sample.

In [ ]:
df_sample = df.get_sample(table_name='buh', sample_percentage=10, overwrite=True)

Two new columns are created in the sample.

In [ ]:
df_sample['root_locatiomn_contexts'] = df_sample.location_stack.json[:1]
df_sample['application'] = df_sample.global_contexts.gc.application
df_sample.head()

Using `.get_unsampled()`, the operations that are done on the sample (the creation of the two columns), are applied to the entire data set:

In [ ]:
df_unsampled = df_sample.get_unsampled()
df_unsampled.head()

The sample can also be used for grouping and aggregating. The example below counts all hits and the unique event_types in the sample:

In [ ]:
df_sample_grouped = df_sample.groupby(['application']).agg({'event_type':'nunique','session_hit_number':'count'})
df_sample_grouped.head()

As can be seen from the counts, unsampling applies the transformation to the entire data set:

In [ ]:
df_unsampled_grouped = df_sample_grouped.get_unsampled()
df_unsampled_grouped.head()

This concludes the Modeling with Bach demo.

We’ve demonstrated a handful of the operations that Bach supports and hope you’ve gotten a taste of what Bach can do for your modeling workflow. 

The full Objectiv Bach API reference is available here: https://objectiv.io/docs/modeling/reference

There is another notebook in the same folder [model-hub-demo-notebook.ipynb](model-hub-demo-notebook.ipynb) that focuses on using the open model hub, demonstrating how you can use Bach to combine pre-built models to quickly answer common product analytics questions.